In [ ]:
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

In [ ]:
import grequests
import uscensus
from uscensus import SqlAlchemyCache

In [ ]:
from configparser import RawConfigParser

In [ ]:
cp = RawConfigParser()

In [ ]:
cp.read('.census')

In [ ]:
import requests
s = requests.Session()

In [ ]:
# Work around for some Windows TLS issues

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.poolmanager import PoolManager
import ssl

class MyAdapter(HTTPAdapter):
    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = PoolManager(num_pools=connections,
                                       maxsize=maxsize,
                                       block=block,
                                       ssl_version=ssl.PROTOCOL_TLSv1_2)
s.mount('https://', MyAdapter())

In [ ]:
cache = SqlAlchemyCache('sqlite://')

In [ ]:
%%time
with cache:
    cl = uscensus.DiscoveryInterface(
        cp.get('census','api_key'),
        cache,
        session=s,
    )

In [ ]:
acs = cl.search('"acs 5-year detailed tables"')
acs

In [ ]:
api = next(iter(acs))
[api.endpoint for api in acs]

In [ ]:
api.endpoint

In [ ]:
api.geographies

In [ ]:
api.groups

In [ ]:
vars = api.searchVariables('"median gross rent"')
vars

In [ ]:
med_rent_by_county = api(['B25064_001E'], geo_for={'county': '*'})
med_rent_by_county.set_index(['state', 'county'], inplace=True)
med_rent_by_county.sort_index(inplace=True)
med_rent_by_county

In [ ]:
api.vintage

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import fiona
import geopandas as gpd
import shapely

In [ ]:
%%time
# Generated in GetCountyShapes.ipynb
counties_df = gpd.GeoDataFrame.from_file('counties.geojson',
                                         driver='GeoJSON')
counties_df.crs = fiona.crs.from_epsg(4326)

In [ ]:
continental = shapely.geometry.box(-125.0011, 24.9493, -66.9326, 49.5904)
counties_df[counties_df.geometry.within(continental)].simplify(0.1).plot(
    figsize=((15,15)),
    linewidth=0.5)

In [ ]:
%time
counties_df.set_index(['STATE', 'COUNTY'], inplace=True)
counties_df.sort_index(inplace=True)

In [ ]:
%time
counties_df.index.names=['state','county']
joined = counties_df.join(med_rent_by_county)

In [ ]:
#joined.total_bounds
continental = shapely.geometry.box(-125.0011, 24.9493, -66.9326, 49.5904)
joined_cont = joined[joined.intersects(continental)]

In [ ]:
axes=joined_cont.plot(column='B25064_001E', 
                      figsize=(15,15),
                      linewidth=0.5)
axes.figure.dpi=190
axes

In [ ]:
joined_cont.head()

In [ ]:
import matplotlib as mpl
import numpy as np
inc_min=joined_cont['B25064_001E'].min()
inc_max=joined_cont['B25064_001E'].max()
X = [[0, 0], [.9, .9]]
plt.gca().imshow(X, interpolation='bicubic', cmap=mpl.cm.get_cmap(),
          extent=(0, .1, 0, 1), alpha=1)